In [ ]:
from paramak import RotateStraightShape

pf_coil = RotateStraightShape(points=[(500, 500),
                                      (550, 500),
                                      (550, 550),
                                      (500, 550)])

pf_coil.solid

In [ ]:
from paramak import RotateMixedShape

center_column = RotateMixedShape(points=[(50, 600, 'straight'),
                                   (150, 600, 'spline'),
                                   (100, 0, 'spline'),
                                   (150, -600, 'straight'),
                                   (50, -600, 'straight'),
                                  ])

center_column.solid

In [ ]:
from paramak import RotateSplineShape

blanket = RotateSplineShape(
    points=[
        (600, 0),
        (600, -20),
        (500, -300),
        (400, -300),
        (400, 0),
        (400, 300),
        (500, 300),
        (600, 20),
        (600, 0),
    ],rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315]
)
blanket.solid

In [ ]:
from paramak import Reactor

my_reactor = Reactor()

blanket.stp_filename = 'blanket.stp'
pf_coil.stp_filename = 'pf_coil.stp'
center_column.stp_filename = 'center_column.stp'

blanket.material_tag = 'blanket_material'
pf_coil.material_tag = 'pf_coil_material'
center_column.material_tag = 'center_column_material'

my_reactor.add_shape(blanket)
my_reactor.add_shape(pf_coil)
my_reactor.add_shape(center_column)

In [ ]:
my_reactor.export_neutronics_description('manifest.json')
my_reactor.export_stp()

In [ ]:
!ls

In [ ]:
!trelis make_faceteted_neutronics_model.py

# or to avoid GUI pop up use  !trelis -batch -nographics make_faceteted_neutronics_model.py

In [ ]:
!make_watertight dagmc_notwatertight.h5m -o dagmc.h5m

In [ ]:
from neutronics_material_maker import Material

mat1 = Material(material_name='Li4SiO4',
                material_tag='blanket_material').openmc_material

mat2 = Material(material_name='copper',
                material_tag='pf_coil_material').openmc_material

mat3 = Material(material_name='WC',
                material_tag='center_column_material').openmc_material

In [ ]:
from parametric_plasma_source import Plasma

my_plasma = Plasma(elongation=2.9,
                   minor_radius=1.118*100,
                   major_radius=1.9*100,
                   triangularity = 0.55)
my_plasma.export_plasma_source('my_custom_plasma_source.so')

In [ ]:
import openmc

mats = openmc.Materials([mat1, mat2, mat3])


In [ ]:
settings = openmc.Settings()
settings.batches = 10
settings.inactive = 0
settings.particles = 1000
settings.run_mode = 'fixed source'
settings.dagmc = True 
settings.source = openmc.Source(library='./my_custom_plasma_source.so')


In [ ]:
tallies = openmc.Tallies()
tbr_tally = openmc.Tally(name='TBR')
tbr_tally.scores = ['(n,Xt)'] # where X is a wild card
tallies.append(tbr_tally)


In [ ]:
universe = openmc.Universe()
geom = openmc.Geometry(universe)

In [ ]:
model = openmc.model.Model(geom, mats, settings, tallies)
statepoint_filename = model.run()

In [ ]:
# open the results file
sp = openmc.StatePoint(statepoint_filename)

# access the tally
tbr_tally = sp.get_tally(name='TBR')
df = tbr_tally.get_pandas_dataframe()
tbr_tally_result = df['mean'].sum()

# print result
print('The tritium breeding ratio was found, TBR = ', tbr_tally_result)